In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").appName("chapter11").getOrCreate()

24/01/05 15:33:09 WARN Utils: Your hostname, quangtn933.local resolves to a loopback address: 127.0.0.1; using 192.168.1.90 instead (on interface en0)
24/01/05 15:33:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/05 15:33:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

In [4]:
data_without_header = spark.read.option("inSchema", True).option("header", False).\
    csv("/Users/quangtn/Desktop/01_work/01_job/02_ml/PySpark/covtype.data")

In [5]:
data_without_header

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string, _c28: string, _c29: string, _c30: string, _c31: string, _c32: string, _c33: string, _c34: string, _c35: string, _c36: string, _c37: string, _c38: string, _c39: string, _c40: string, _c41: string, _c42: string, _c43: string, _c44: string, _c45: string, _c46: string, _c47: string, _c48: string, _c49: string, _c50: string, _c51: string, _c52: string, _c53: string, _c54: string]

In [6]:
colnames = ["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
            "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
            "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", "Horizontal_Distance_To_Fire_Points"] + \
[f"Wilderness_Area_{i}" for i in range(4)]+ [f"Solid_Type_{i}" for i in range(40)] + ["Cover_Type"]

In [7]:
data = data_without_header.toDF(*colnames).withColumn("Cover_Type", col("Cover_Type").cast(DoubleType()))

In [8]:
from pyspark.sql.functions import col

#table = spark.sql("table")

data = data.select([col(c).cast("double") for c in data.columns])

In [9]:
data

DataFrame[Elevation: double, Aspect: double, Slope: double, Horizontal_Distance_To_Hydrology: double, Vertical_Distance_To_Hydrology: double, Horizontal_Distance_To_Roadways: double, Hillshade_9am: double, Hillshade_Noon: double, Hillshade_3pm: double, Horizontal_Distance_To_Fire_Points: double, Wilderness_Area_0: double, Wilderness_Area_1: double, Wilderness_Area_2: double, Wilderness_Area_3: double, Solid_Type_0: double, Solid_Type_1: double, Solid_Type_2: double, Solid_Type_3: double, Solid_Type_4: double, Solid_Type_5: double, Solid_Type_6: double, Solid_Type_7: double, Solid_Type_8: double, Solid_Type_9: double, Solid_Type_10: double, Solid_Type_11: double, Solid_Type_12: double, Solid_Type_13: double, Solid_Type_14: double, Solid_Type_15: double, Solid_Type_16: double, Solid_Type_17: double, Solid_Type_18: double, Solid_Type_19: double, Solid_Type_20: double, Solid_Type_21: double, Solid_Type_22: double, Solid_Type_23: double, Solid_Type_24: double, Solid_Type_25: double, Solid_T

In [10]:
(train_data, test_data) = data.randomSplit([0.9, 0.1])

In [11]:
input_cols = colnames[:-1]

In [12]:
vector_assembler = VectorAssembler(inputCols=input_cols, outputCol="featureVector")
classifier = DecisionTreeClassifier(seed=1234, labelCol="Cover_Type",\
                                    featuresCol="featureVector",predictionCol="prediction")

In [13]:
pipeline = Pipeline(stages=[vector_assembler, classifier])

In [14]:
import mlflow
import mlflow.spark
import pandas as pd
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [15]:
with mlflow.start_run(run_name="decision-tree"):
    # log param: max_depth
    mlflow.log_param("max_depth", classifier.getMaxDepth())
    # Log model
    pipeline_model = pipeline.fit(train_data)
    mlflow.spark.log_model(pipeline_model, "model")
    # Log metrics: Accuracy and F1
    pred_df = pipeline_model.transform(test_data)
    evaluator = MulticlassClassificationEvaluator(labelCol="Cover_Type", predictionCol="prediction")
    accuracy = evaluator.setMetricName("accuracy").evaluate(pred_df)
    f1 = evaluator.setMetricName("f1").evaluate(pred_df)
    mlflow.log_metrics({"accuracy": accuracy, "f1": f1})
    # Log artifact: feature importance scores.
    tree_model = pipeline_model.stages[-1]
    feature_importance_df = (pd.DataFrame(list(zip(vector_assembler.getInputCols(), \
        tree_model.featureImportances)), columns=["feature", "importance"]).sort_values(by="importance", \
        ascending=False))
    feature_importance_df.to_csv("feature-importance.csv", index=False)
    mlflow.log_artifact("feature-importance.csv")

24/01/05 15:33:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/01/05 15:33:28 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
/Users/quangtn/opt/anaconda3/envs/bentoML/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [16]:
!mlflow ui

/Users/quangtn/opt/anaconda3/envs/bentoML/lib/python3.8/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
/Users/quangtn/opt/anaconda3/envs/bentoML/bin/python: No module named gunicorn.__main__; 'gunicorn' is a package and cannot be directly executed
Running the mlflow server failed. Please see the logs above for details.


## Calling model through API by MLflow.

$ mlflow models serve --model-uri runs:/0433bb047f514e28a73109bbab767222/model -p 7000

In [17]:
import requests

In [18]:
host= "0.0.0.0"
port = "7001"

In [19]:
url = f'http://{host}:{port}/invocations'

In [23]:
headers = {
    'Content-Type': 'application/json;',
    'format': 'pandas-split'
}

In [33]:
http_data = '{"columns": ["Elevation", "Aspect", "Slope",\
    "Horizontal_Distance_To_Hydrology",\ 
    "Vertical_Distance_To_Hydrology","Horizontal_Distance_To_Roadways", \ 
    "Hillshade_9am","Hillshade_Noon","Hillshade_3pm",\ 
    "Horizontal_Distance_To_Fire_Points",\ 
    "Wilderness_Area_0","Wilderness_Area_1","Wilderness_Area_2",\ 
    "Wilderness_Area_3","Soil_Type_0","Soil_Type_1","Soil_Type_2",\ 
    "Soil_Type_3","Soil_Type_4","Soil_Type_5","Soil_Type_6",\ 
    "Soil_Type_7","Soil_Type_8","Soil_Type_9","Soil_Type_10",\ 
    "Soil_Type_11","Soil_Type_12","Soil_Type_13", \ 
    "Soil_Type_14","Soil_Type_15","Soil_Type_16", \ 
    "Soil_Type_17","Soil_Type_18","Soil_Type_19", \
    "Soil_Type_20","Soil_Type_21","Soil_Type_22", \ 
    "Soil_Type_23","Soil_Type_24","Soil_Type_25", \ 
    "Soil_Type_26","Soil_Type_27","Soil_Type_28", \ 
    "Soil_Type_29","Soil_Type_30","Soil_Type_31", \ 
    "Soil_Type_32","Soil_Type_33","Soil_Type_34", \ 
    "Soil_Type_35","Soil_Type_36","Soil_Type_37", \ 
    "Soil_Type_38","Soil_Type_39","Cover_Type"], \ 
    "index":[0],\
    "data":[[2596,51,3,258,0,510,221,232,148,6279,1,\
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,\ 
        0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5.0]] }'

SyntaxError: EOL while scanning string literal (1816678397.py, line 2)

In [35]:
r =requests.post(url = url ,headers= headers, data=http_data)
print(f'Predictions: {r.text}')

NameError: name 'http_data' is not defined